In [34]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
#Funcion para arrancar el webdriver
def iniciar_wd(headless=False):
    options=uc.ChromeOptions()
    options.add_argument('--password-store=basic')
    options.add_experimental_option(
        'prefs',
        {'credentials_enable_service':False,
         'profile.password_manager_enabled':False}
    )
    if headless:
        options.add_argument('--headless')

    #Iniciamos el driver
    driver=uc.Chrome(options=options,headless=headless,log_level=3)
    driver.maximize_window()
    
    return driver

In [107]:
driver=iniciar_wd()
driver.get('https://www.airbnb.es')
wait=WebDriverWait(driver,30)

In [136]:
opc=['A pie de playa', 'En el campo','Cabañas']
for o in opc:
    xpath_expresion = f"//*[contains(text(), '{o}')]"

    # Buscar el elemento que contiene el texto
    elemento = driver.find_element(By.XPATH,xpath_expresion)
    elemento.click()
    time.sleep(5)

In [137]:
scroll_height = driver.execute_script("return document.body.scrollHeight;")
actions = ActionChains(driver)
# Hacer scroll hasta el final de la página
driver.execute_script(f"window.scrollTo(0, {scroll_height});")

for i in range(6):
    actions.send_keys(Keys.PAGE_UP).perform()
    time.sleep(.1)

In [138]:
xpath_expresion = f"//button"

buts=driver.find_elements(By.XPATH,xpath_expresion)
for b in buts:
    try:
        if b.text.find('Mostrar más')!=-1:
            b.click()
    except:
        pass

In [139]:
actions = ActionChains(driver)
for i in range(120):
    actions.send_keys(Keys.PAGE_DOWN).perform()
    time.sleep(.1)

time.sleep(2)


In [131]:
html=driver.page_source
soup=BeautifulSoup(html, 'html.parser')
res_playa=[]
urls=soup.find_all('a')
for u in urls:
    en=u.get('href')
    
    if en.find('/rooms')!=-1:
        res_playa.append(f'https://www.airbnb.es{en}')


In [132]:
len(res_playa)

1817

In [159]:
def obt_url():
    driver=iniciar_wd()
    driver.get('https://www.airbnb.es')

    time.sleep(3)
    result={}
    opc=['A pie de playa', 'En el campo','Cabañas']
    for o in opc:
        xpath_expresion = f"//*[contains(text(), '{o}')]"

        # Buscar el elemento que contiene el texto
        elemento = driver.find_element(By.XPATH,xpath_expresion)
        elemento.click()
        time.sleep(2)

        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        actions = ActionChains(driver)
        # Hacer scroll hasta el final de la página
        driver.execute_script(f"window.scrollTo(0, {scroll_height});")
        actions.send_keys(Keys.PAGE_UP).perform()
        time.sleep(3)
        for i in range(6):
            actions.send_keys(Keys.PAGE_UP).perform()
            time.sleep(.5)
        time.sleep(5)
        
        xpath_expresion = f"//button"
        buts=driver.find_elements(By.XPATH,xpath_expresion)
        for b in buts:
            try:
                if b.text.find('Mostrar más')!=-1:
                    b.click()
                    print('HA ENCONTRADO EL BOTON!!')
            except:
                print('NO TIENE ATRIBUTO TEXT')
        time.sleep(3)
        for i in range(120):
            actions.send_keys(Keys.PAGE_DOWN).perform()
            time.sleep(.1)
        time.sleep(2)
        
        
        html=driver.page_source
        soup=BeautifulSoup(html, 'html.parser')
        res=[]
        urls=soup.find_all('a')
        for u in urls:
            en=u.get('href')
            
            if en.find('/rooms')!=-1:
                res.append(f'https://www.airbnb.es{en}')
        
        result[o]=len(res)
    return result

In [160]:
res=obt_url()
res

HA ENCONTRADO EL BOTON!!
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT
NO TIENE ATRIBUTO TEXT


{'A pie de playa': 1750, 'En el campo': 1849, 'Cabañas': 1881}